In [29]:
from skimage import io
from pyxelate import Pyx, Pal
from PIL import Image, ImageEnhance 
import pandas as pd





In [30]:
# Load the data
data = pd.read_feather('data/temp/data.feather')

# Data games
data_games = pd.read_csv('data/v1/games-data.csv')

# Find the games ("name") with the most submissions, and keep the top 100, along with their release date and platform
top_games = data[["name", "name_category"]].value_counts()

# Convert the index to columns
top_games = top_games.reset_index()

# For each game, find the first release date and platform
top_games_data = data[data["name"].isin(top_games["name"])].groupby(["name", "name_category"]).agg(
    {"releaseDate": "first", "platform_name": "first"}
)


# Add count of submissions for each game and category in the top games
top_games_data = top_games_data.join(data[["name", "name_category"]].value_counts(), on=["name", "name_category"])



# Add sum of all submissions for each game in the top games
top_games_data["sum_count"] = top_games_data["count"].groupby("name").transform("sum")



# Only keep the top 3 count of submissions for each game...
# First sort by count of submissions (descending) for each game
top_games_data = top_games_data.sort_values("count", ascending=False)


# Sort by count of submissions (descending)
top_games_data = top_games_data.sort_values(["sum_count", "count"], ascending=False)

# Remove any rown that have NaN or None
top_games_data = top_games_data.dropna()

# Reset the index
top_games_data = top_games_data.reset_index()


top_50_games_data_names = top_games_data.drop_duplicates(subset=["name"]).head(50)

# Do a inner join with the data_games to get the game_id
top_50_games_data = pd.merge(top_50_games_data_names, data_games, on='name')

In [31]:
for id in top_50_games_data['ID']:
    c = Image.open(f"/Users/cdroin/Library/CloudStorage/GoogleDrive-colasdroin@gmail.com/Mon Drive/Data_viz/speedruns/app-v2/public/images/{id}_icon.webp")
    d = c.resize((c.size[0]*5, c.size[1]*5), Image.HAMMING, reducing_gap=5)
    e = ImageEnhance.Sharpness(d).enhance(5)
    e.save(f"/Users/cdroin/Library/CloudStorage/GoogleDrive-colasdroin@gmail.com/Mon Drive/Data_viz/speedruns/app-v2/public/images/{id}_icon_rescaled.webp")
